In [ ]:
import numpy as np
from numpy import *
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from os import listdir
from os.path import isfile, join, isdir

In [ ]:
#from plot_results import get_dbl_metrics
import sys
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import precision_recall_curve, accuracy_score, average_precision_score

import matplotlib.pyplot as plt

def get_dbl_metrics(test, score):
	fpr, tpr, thresholds = metrics.roc_curve(test, score, pos_label=1)
	rauc = metrics.auc(fpr, tpr)

	precision, recall, thresholds = precision_recall_curve(test, score, pos_label=1)
	prauc = metrics.auc(recall, precision)
	
	ap = average_precision_score(test, score)
	return rauc, prauc, ap

In [ ]:
save_path = '../results_manuscript/heatmaps/'


In [ ]:
path = '../results_benchmark/'
files = [f.split('_') for f in listdir(path) if (isfile(join(path, f)) & (f[-3:]=='csv'))]
files = np.array(files)
files

In [ ]:
data_names = np.sort(np.unique(files[:,0]))
methods = np.unique(files[:,1])

In [ ]:
tmp1 = np.zeros((len(methods), len(data_names)))
tmp2 = np.zeros((len(methods), len(data_names)))
hm_pr = pd.DataFrame(tmp1, index=methods, columns=np.sort(data_names))
hm_roc = pd.DataFrame(tmp2, index=methods, columns=np.sort(data_names))

tmp3 = np.zeros((1, len(data_names)))
weights = pd.DataFrame(tmp3, index=['number of cells'], columns=np.sort(data_names))

In [ ]:
for file in files:
    res = pd.read_csv(path + '_'.join(file))
    
    ano_path  = '../data/mtx_files/' + file[0] + '_anno.csv'

    #- READ IN BARCODE ANNOTATIONS
    ano = pd.read_csv(ano_path)
    true = pd.factorize(ano.x)[0]
    labels = ano.x
    if (labels[0]=='doublet'):
        tmp = true + 3
        tmp[tmp==3] = 1
        tmp[tmp==4] = 0
        true = tmp
    
    score_roc, score_pr, _ = get_dbl_metrics(true, res['doublet_scores'])
    plt.close()
    
    method = file[1]
    data_name = file[0]
    
    weights[data_name].loc['number of cells'] = len(true)
        
    hm_roc[data_name].loc[method] = score_roc
    hm_pr[data_name].loc[method] = score_pr
    

In [ ]:
hm_roc = hm_roc.replace(0,np.nan)
hm_roc = hm_roc.iloc[np.argsort(-1*hm_roc.mean(axis=1)),:]
hm_roc

In [ ]:
hm_pr = hm_pr.replace(0,np.nan)
hm_pr = hm_pr.iloc[np.argsort(-1*hm_pr.mean(axis=1)),:]
hm_pr

In [ ]:
hm_roc.to_csv(save_path + 'ROC.csv') 
hm_pr.to_csv(save_path + 'PR.csv') 

In [ ]:
def mk_heatmap(hm, title = 'Heatmap'):

    hm_norm = hm.rank(axis=1, method='max')

    fig, (ax,ax2) = plt.subplots(ncols=1,nrows=2,gridspec_kw=dict(height_ratios=[16,1]),
                                 figsize=(0.9*5,0.9*4.5),
                                 dpi=300)
    
    sns.heatmap(hm_norm, square=False, ax=ax, cbar=False, 
                annot=hm, cmap="YlOrRd",  fmt='g', 
                xticklabels = False, annot_kws={'size': 8}, yticklabels=True)#, cbar_kws=cbar_kws) #YlOrRd
    ax.set_yticklabels(ax.get_yticklabels(),rotation=45)
    ax.tick_params(axis="y", labelsize=8)
    ax.tick_params(bottom=False)

    
    
    hm = np.round(pd.DataFrame(hm.mean(axis=0)).T, 1) 
    hm.index = ['mean']

    sns.heatmap(hm, square=False, cbar=False, annot=hm, cmap="YlOrRd",  fmt='g',
               xticklabels=True, yticklabels=True, annot_kws={'size': 8})#, cbar_kws=cbar_kws) #YlOrRd
    ax2.set_yticklabels(ax2.get_yticklabels(),rotation=45)
    ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45, horizontalalignment='right')
    ax2.tick_params(axis="y", labelsize=8)
    #a.xticks(rotation=45, horizontalalignment='right')
    ax2.tick_params(axis="x", labelsize=10)

    ax.set_title(title, fontsize=18)
    
    plt.tight_layout()
    fig.subplots_adjust(hspace=0.05)
    fig.subplots_adjust(wspace=10)

In [ ]:
save_path

In [ ]:
mk_heatmap(np.round(hm_roc.T*100,0), title = 'Area Under ROC')
plt.savefig(save_path + 'ROC_heatmap.png', dpi=300)
mk_heatmap(np.round(hm_pr.T*100,0), title = 'Area Under PRC')
plt.savefig(save_path + 'PRC_heatmap.png', dpi=300)


In [ ]:
hm = np.round(hm_pr.T*100,0)
hm_norm = hm.rank(axis=1, method='max')

fig, (ax,ax2) = plt.subplots(ncols=1,nrows=2,gridspec_kw=dict(height_ratios=[16,1]),
                             figsize=(0.9*5,0.9*4.5),
                             dpi=300)

p=sns.heatmap(hm, square=False, ax=ax, cbar=True, cmap="YlOrRd",  fmt='g', 
            xticklabels = False, annot_kws={'size': 8}, yticklabels=True)#, cbar_kws=cbar_kws) #YlOrRd

plt.tight_layout()
plt.savefig(save_path + 'PRC_heatmap_CBAR.png', dpi=300)


In [ ]:
w = weights.divide(int(np.sum(weights, axis=1)))
weighted_average = hm_roc.multiply(w.T['number of cells'], axis=1).sum(axis=1)

In [ ]:
plt.figure(figsize=(4,.25),  dpi=300)

hm = np.round(pd.DataFrame(weighted_average).T *100, 1) 
hm.index = ['weighted average']

ax=sns.heatmap(hm, square=False, cbar=False, annot=hm, cmap="YlOrRd",  fmt='g')#, cbar_kws=cbar_kws) #YlOrRd
ax.set_yticklabels(ax.get_yticklabels(),rotation=45)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_title('ROC Mean', fontsize=12)

plt.tight_layout()

In [ ]:
w = weights.divide(int(np.sum(weights, axis=1)))
weighted_average = hm_pr.multiply(w.T['number of cells'], axis=1).sum(axis=1)

In [ ]:
plt.figure(figsize=(4,.25),  dpi=300)

hm = np.round(pd.DataFrame(weighted_average).T *100, 1) 
hm.index = ['weighted average']

ax=sns.heatmap(hm, square=False, cbar=False, annot=hm, cmap="YlOrRd",  fmt='g')#, cbar_kws=cbar_kws) #YlOrRd
ax.set_yticklabels(ax.get_yticklabels(),rotation=45)
ax.set_xticklabels(ax.get_xticklabels(),rotation=90)
ax.set_title('PRC Mean', fontsize=12)

plt.tight_layout()